In [ ]:
# Code from https://github.com/parth647/reddit_scraping_using_praw/blob/master/python_reddit_scrapy.py
# Code from https://github.com/D3vd/Reddit_Image_Scraper/blob/master/Reddit_image_scraper.py

In [ ]:
import json
import re
import praw
import os
import requests
import pandas as pd
import configparser
from prawcore.exceptions import ResponseException

from reddit_image_scrapper import get_img_subreddit, delete_img_list

In [ ]:
def get_client_info():
  config = configparser.ConfigParser()
  config.read("config.ini")
  id = config["ALPHA"]["client_id"]
  secret = config["ALPHA"]["client_secret"]

  return id, secret

In [ ]:
# Acessing the reddit api

clien_id, client_secret = get_client_info()
reddit = praw.Reddit(client_id=clien_id,#my client id
                     client_secret=client_secret,  #your client secret
                     user_agent='retrieve kb data', #user agent name
                     username = '',     # your reddit username
                     password = '')     # your reddit password

sub = ['MechanicalKeyboards']  # make a list of subreddits you want to scrape the data from
# Alternate sub to mine: 'MechanicalKeyboards'

## To scrape stickied comments and pictures of the reddit post
main file -> json with the following format
{'sticky_comment': raw_text, 'title': raw_text, 'img_path': [path_to_saved_images]}

In [ ]:
list_scraped_dict = []

delete_img_list() # Delete image list to get clean slate
os.makedirs('data', exist_ok=True)

try:
  for i, submission in enumerate(reddit.subreddit('MechGroupBuys').new(limit=2000)):    
    
    title_pattern = '\[(?:GB|Pre-order|Preorder|In-stock)\](.*)/' #Bring text eda code to help save files
    try:
      title = re.findall(title_pattern, submission.title.replace('\n', ''))[0].replace('/', '').strip()
    except:
      title = submission.title.replace('[GB]', '').strip()
      title = title[:title.find('//')]
  
    
    if os.path.exists(os.path.join('data', f'scrapped_data_{title}.json')): continue
    print(submission.title)
    for top_level_comment in submission.comments:
      if top_level_comment.stickied:        
        
        # If the key is to extract the price, try to parse all rows into text file then manipulate from there
        #print(top_level_comment.body)
        sticky_comment = top_level_comment.body
        
    list_file_loc = get_img_subreddit(submission, num_img_limit=50, sleep_interval=1)
    scraped_dict = {'title': submission.title, 'sticky_comment': sticky_comment, 'img_paths': list_file_loc}
            
    with open(os.path.join('data', f'scrapped_data_{title}.json'), 'w') as json_output:
      json.dump(scraped_dict, json_output, indent=2)

    #list_scraped_dict.append(scraped_dict)
      
  #with open("scrapped_data.json", "w") as json_output:
   #json.dump(list_scraped_dict, json_output, indent=2)
  
          
except ResponseException as e:
  pass

In [ ]:
# try:
#   #for submission in reddit.subreddit("MechanicalKeyboards").new(limit=10):
#   submissions = reddit.subreddit("MechGroupBuys").new(limit=10)
#   #for submission in submissions: print(submission.url)

#   # The important thing is the code now works
#   get_img_subreddit(submission, num_img_limit=50, sleep_interval=3)

# except ResponseException as e:
#   pass

### Code below this cell is not in use for now

In [ ]:


########################################
#   CREATING DICTIONARY TO STORE THE DATA WHICH WILL BE CONVERTED TO A DATAFRAME
########################################

#   NOTE: ALL THE POST DATA AND COMMENT DATA WILL BE SAVED IN TWO DIFFERENT
#   DATASETS AND LATER CAN BE MAPPED USING IDS OF POSTS/COMMENTS AS WE WILL 
#   BE CAPTURING ALL IDS THAT COME IN OUR WAY

# SCRAPING CAN BE DONE VIA VARIOUS STRATEGIES {HOT,TOP,etc} we will go with keyword strategy i.e using search a keyword
# query = ['Gaming']

# for item in query:
#     post_dict = {
#         "title" : [],
#         "score" : [],
#         "id" : [],
#         "url" : [],
#         "comms_num": [],
#         "created" : [],
#         "body" : []
#     }
#     comments_dict = {
#         "comment_id" : [],
#         "comment_parent_id" : [],
#         "comment_body" : [],
#         "comment_link_id" : []
#     }
#     for submission in subreddit.search(query,sort = "top",limit = 1):
#         post_dict["title"].append(submission.title)
#         post_dict["score"].append(submission.score)
#         post_dict["id"].append(submission.id)
#         post_dict["url"].append(submission.url)
#         post_dict["comms_num"].append(submission.num_comments)
#         post_dict["created"].append(submission.created)
#         post_dict["body"].append(submission.selftext)

#         ##### Acessing comments on the post
#         submission.comments.replace_more(limit = 1)
#         for comment in submission.comments.list():
#             comments_dict["comment_id"].append(comment.id)
#             comments_dict["comment_parent_id"].append(comment.parent_id)
#             comments_dict["comment_body"].append(comment.body)
#             comments_dict["comment_link_id"].append(comment.link_id)

#     post_comments = pd.DataFrame(comments_dict)

#     post_comments.to_csv(s+"_comments_"+ item +"subreddit.csv")
#     post_data = pd.DataFrame(post_dict)
#     post_data.to_csv(s+"_"+ item +"subreddit.csv")
